# Imports 

In [ ]:
%matplotlib widget
import numpy as np
import tensorflow as tf
from tensorflow import keras

import mne

from matplotlib import pyplot as plt

from tqdm.autonotebook import tqdm

# Load MNIST Dataset

In [ ]:
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
# x_train = x_train[..., 5:-5]
x_train, x_test = x_train / 255.0, x_test / 255.0

TRAIN_WIDTH = x_train.shape[-1]
N_CHANNELS = 28
N_CLASSES = 10

In [ ]:
labels_int = np.arange(N_CLASSES)
labels = labels_int.astype(str)
labels, labels_int

In [ ]:
x_train.shape, x_test.shape

In [ ]:
dig_idx = 2000


plt.figure()
plt.imshow(x_train[dig_idx])
plt.title(y_train[dig_idx])

## Prepend Baseline

In [ ]:
x_train = np.concatenate([np.zeros_like(x_train), 
                          x_train], 
                         axis=-1)

In [ ]:
plt.figure()
plt.imshow(x_train[0])
plt.title(y_train[0])
plt.axvline(TRAIN_WIDTH-0.5, color='r')

plt.figure()
plt.imshow(x_test[0])
plt.title(y_test[0])

# Mimic EMG envelope channels

In [ ]:
emg_train = x_train.transpose(0,2,1).reshape(-1, N_CHANNELS).T
emg_test = x_test.transpose(0,2,1).reshape(-1, N_CHANNELS).T
'train', emg_train.shape, 'test', emg_test.shape

In [ ]:
plt.figure(figsize=(12,6))
plt.imshow(emg_train[:, :10*TRAIN_WIDTH])
plt.vlines([TRAIN_WIDTH-0.5 + TRAIN_WIDTH*i*2 for i in range(5)], *plt.ylim(), color='r')
plt.xlabel('Timestamps')
plt.ylabel('Channels')
plt.title('EMG channels')

# Add some noise

In [ ]:

emg_train += np.random.random(size = (emg_train.shape)) * 0.33
emg_test  += np.random.random(size = (emg_test.shape)) * 0.33

In [ ]:
plt.figure(figsize=(12,6))
plt.imshow(emg_train[:, :10*TRAIN_WIDTH])
plt.vlines([TRAIN_WIDTH-0.5 + TRAIN_WIDTH*i*2 for i in range(5)], *plt.ylim(), color='r')
plt.xlabel('Time, ts')
plt.ylabel('Channels')
plt.title('Mimicking EMG envelope with noise')

In [ ]:
plt.figure(figsize=(12,6))
plt.imshow(emg_test[:, :10*TRAIN_WIDTH])
# plt.vlines([TRAIN_WIDTH-0.5 + TRAIN_WIDTH*i*2 for i in range(5)], *plt.ylim(), color='r')
plt.xlabel('Time, ts')
plt.ylabel('Channels')
plt.title('Mimicking TEST EMG envelope with noise')

# Create raw MNE object

In [ ]:
raw_test  = mne.io.RawArray(emg_test, mne.create_info(list(np.arange(N_CHANNELS).astype(str)), sfreq=28, ch_types='emg'))
raw_train = mne.io.RawArray(emg_train, mne.create_info(list(np.arange(N_CHANNELS).astype(str)), sfreq=28, ch_types='emg'))


## Create event data

In [ ]:
events_train = np.stack([[TRAIN_WIDTH + TRAIN_WIDTH*2*i for i in range(len(x_train))], 
                         np.zeros(len(x_train), 
                                  dtype=int), y_train], axis=1)
events_train

## Visualize

In [ ]:
raw_train.plot(scalings={'emg':1}, 
               n_channels=N_CHANNELS, 
               events=events_train);

plt.gcf().set_size_inches(12,6)

plt.tight_layout()

## NO-GO Event

NO-GO selection strategy 
- take random events from the train set - assume random events result in random outputs
- derive from inference, e.g. NO_GO == below threshold activation
- make explicit NO-GO event in the data acquisition protocol, e.g. from baseline
- construct NO-GO events from the data

In [ ]:
%matplotlib widget

In [ ]:
plt.figure()
plt.hist(y_train, bins=np.linspace(-0.25, 9.25, 20))
plt.title('Label representation')

In [ ]:
# assume homogeneous label distribution

n_nogo_labels = len(y_train)//N_CLASSES
n_nogo_labels

### Random Strategy (you can try it on your own)

In [ ]:
nogo_timestamps = np.random.choice(np.arange(len(raw_train.times)), n_nogo_labels)
events_nogo = np.stack([nogo_timestamps, np.zeros_like(nogo_timestamps), np.ones_like(nogo_timestamps)*10], axis=1)
events_nogo

In [ ]:
## Inject NOGO events

# concatenation with sorting on the go
events_train_w_nogo_random = np.array(sorted(np.concatenate([events_train, events_nogo], 
                                                             axis=0), 
                                              key=lambda row: row[0]))


## Construction of NO-GO events strategy (see supplementary presentation)

How many timestamps to slice? 
SampleWidth * Num_NOGO_events = TRAIN_WIDTH*6000 = 

In [ ]:
TRAIN_WIDTH*6000

In [ ]:
np.random.choice(np.arange(len(raw_train.times)), size=TRAIN_WIDTH*6000, replace=False)

In [ ]:
raw_train._data[:,np.random.choice(np.arange(len(raw_train.times)), size=TRAIN_WIDTH*6000, replace=False)].shape

In [ ]:
raw_train._data[:,np.random.choice(np.arange(len(raw_train.times)), size=TRAIN_WIDTH*6000, replace=False)].reshape(28, TRAIN_WIDTH, -1).shape

In [ ]:
nogo_constructed = raw_train._data[:,np.random.choice(np.arange(len(raw_train.times)), size=TRAIN_WIDTH*6000, replace=False)].reshape(28, -1, TRAIN_WIDTH)

emg_nogo = np.concatenate([np.zeros_like(nogo_constructed) + np.random.random(size = (nogo_constructed.shape)) * 0.33, nogo_constructed], axis=-1).reshape(28, -1)

plt.figure(figsize=(12, 6))
plt.imshow(emg_nogo[:, :10*TRAIN_WIDTH])
plt.vlines([TRAIN_WIDTH + TRAIN_WIDTH*2*i for i in range(5)], *plt.ylim(), color='r')
plt.tight_layout()

emg_nogo.shape

In [ ]:
emg_train_w_nogo = np.concatenate([emg_train, emg_nogo], axis=-1)
emg_train_w_nogo.shape

In [ ]:
raw_train_w_nogo  = mne.io.RawArray(emg_train_w_nogo, mne.create_info(list(np.arange(N_CHANNELS).astype(str)), sfreq=28, ch_types='emg'))

## Visualize w nogo


In [ ]:
raw_train_w_nogo.plot(scalings={'emg':1},
                       n_channels=N_CHANNELS, 
                       events=events_train);

plt.gcf().set_size_inches(12,6)

plt.tight_layout()

In [ ]:
events_train

In [ ]:
events_nogo = events_train[:n_nogo_labels].copy()
events_nogo[:, 0] += emg_train.shape[-1]
events_nogo[:, -1] = 10
events_nogo

In [ ]:
events_train_w_nogo = np.concatenate([events_train, events_nogo], axis=0)

raw_train_w_nogo.plot(scalings={'emg':1},
                       n_channels=N_CHANNELS, 
                       events=events_train_w_nogo);

plt.gcf().set_size_inches(12,6)

plt.tight_layout()

# Epoch analysis

In [ ]:
sfreq = raw_train_w_nogo.info['sfreq']
epochs = mne.Epochs(raw_train_w_nogo,
                    events = events_train_w_nogo, 
                    tmin=-TRAIN_WIDTH/sfreq, tmax=TRAIN_WIDTH/sfreq, 
                    preload=True)

In [ ]:
vis_label = "5"

av = epochs[vis_label].average(picks=['emg']);
av.plot();
plt.axvline(0, color='r')
plt.tight_layout()

plt.figure()
plt.imshow(av.data, vmin=0, vmax=1, extent=[av.times.min(), av.times.max(), 28, 0], aspect='auto')
plt.title(vis_label)
plt.axvline(0, color='r')
plt.tight_layout()

# Visualize manifold

In [ ]:
epochs._data.shape

In [ ]:
EPOCH_SIZE = len(epochs.times)

In [ ]:
%%time

decimation = 10 # 10 for faster computing
X = epochs._data.reshape(len(epochs), -1)[epochs.events[:, -1]<10][::decimation] # taking all 10 will result in spoiling the resulting figure
y = epochs[epochs.events[:, -1]<10][::decimation].events[:,-1]

X.shape, y.shape

from sklearn.manifold import TSNE
tsne = TSNE(init='pca', verbose=10)
tv = tsne.fit_transform(X)

In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(*(tv.T), 
            c = y, 
            s=2, 
            alpha=0.5, 
            cmap='jet')

h = plt.colorbar(label='digits')

h.set_ticks(np.arange(10))

for l in np.unique(y):
    mask = y==l
    t = plt.text(*tv[mask].mean(axis=0), l, ha='center', va='center', fontsize=20)
    t.set_bbox(dict(facecolor='white', alpha=0.75, edgecolor='black'))

plt.tight_layout()


# Model construction

## Simple RNN

In [ ]:
EPOCH_SHAPE[-1]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Input, LSTM
from tensorflow.keras.utils import to_categorical, plot_model

N_HIDDEN = 50
EPOCH_SHAPE = epochs._data[0].T.shape # in keras time goes first
N_CLASSES = 11

model = Sequential()

model_input = x = Input(shape=[None, 
                               EPOCH_SHAPE[-1]], 
                        batch_size=None)

x = SimpleRNN(N_HIDDEN, 
             activation='relu',
             recurrent_regularizer=tf.keras.regularizers.L1L2(l1=1e-2, l2=1e-2),
             return_sequences=True)(x)

# see comments in supplementary
# x = SimpleRNN(N_HIDDEN, 
#              activation='relu',
#              recurrent_regularizer=tf.keras.regularizers.L1L2(l1=1e-2, l2=1e-2),
#              return_sequences=True)(x)

model_output = tf.keras.layers.TimeDistributed(Dense(units=N_CLASSES, 
                                                     activation='sigmoid'))(x)

model = tf.keras.Model(model_input, model_output)

model.summary(expand_nested=True)

In [ ]:
plot_model(model)

In [ ]:
model.compile(loss='mse', 
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
X = epochs._data.copy()
X = np.transpose(X, [0, 2, 1]) # because timestamps goes first for keras recurrent layers, batch.shape = (Batch_size, Timestamps, Channel1, Channel2, Channel3, ...)
y = to_categorical(epochs.events[:,-1])
y = np.repeat(y[:, np.newaxis, :], 
              axis=1,
              repeats=len(epochs.times))
y[:, epochs.times<0, :] = [[[0]*10+[1]]]

In [ ]:
plt.figure()
plt.imshow(y[0].T)

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
X_t, X_v, y_t, y_v = train_test_split(X, y, stratify=epochs.events[:,-1], test_size=0.1)

In [ ]:
X_t.shape, X_v.shape

In [ ]:
%%time
model.fit(X_t, y_t, epochs=50, validation_data=(X_v, y_v))

In [ ]:
fig, axx = plt.subplots(2,1,sharex=True)
plt.sca(axx[0])
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title('Loss')
plt.sca(axx[1])
plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['val_accuracy'])
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.tight_layout()

In [ ]:
# model.save('model.h5')
model = keras.models.load_model('model.h5')

### Sanity check

In [ ]:
sample_of_interest = 50;
sample = np.transpose(epochs[sample_of_interest]._data, [0, 2, 1])
print('Sample shape', sample.shape)

activations = model.predict(sample)

fig, axx = plt.subplots(3, 1, figsize=(6,6), sharex=True)
plt.sca(axx[0])
plt.imshow(sample[0].T, aspect='auto')
plt.axvline(TRAIN_WIDTH, color='r')
plt.title('Sample')
plt.colorbar()

plt.sca(axx[1])
plt.imshow(y[sample_of_interest].T, aspect='auto')
plt.yticks(np.arange(11), list(range(10))+['NOGO'])
# plt.grid(True)
plt.axvline(TRAIN_WIDTH, color='r')
plt.title('Actual')
plt.colorbar()

plt.sca(axx[2])
plt.imshow(activations[0].T, aspect='auto', vmin=0, vmax=1)
plt.axvline(TRAIN_WIDTH, color='r')
# plt.grid(True)
plt.yticks(np.arange(11), list(range(10))+['NOGO'])
plt.colorbar()

plt.title('Predicted')
plt.tight_layout()

## Sliding inference

In [ ]:
emg_test.shape

In [ ]:
activations_keras = model.predict(emg_test.T[np.newaxis, ...])[0].T.astype(float)

In [ ]:
plt.figure()
plt.imshow(activations_keras[:, 0:1000], 
           aspect='auto', 
           interpolation="nearest", vmin=0, vmax=1)

In [ ]:
y_test[:100]

In [ ]:
activations_keras.shape

In [ ]:
plt.figure()
plt.hist(activations_keras[activations_keras>0], bins=np.arange(0, 1, 0.01));
# plt.xscale('log')

In [ ]:
activation_thresh = (activations_keras[activations_keras>0].mean(-1) + 3*activations_keras[activations_keras>0].std(-1)).mean()
activation_thresh

In [ ]:
filtered_activations = activations_keras.copy()

mne.filter.filter_data(filtered_activations, sfreq=sfreq, l_freq=None, h_freq=2, copy=False, phase='minimum')

activation_thresh = (filtered_activations[filtered_activations>0].mean(-1) + 3*filtered_activations[filtered_activations>0].std(-1)).mean()
# activation_thresh = 0

plt.figure()
plt.hist(filtered_activations[filtered_activations>0], bins = np.arange(0, 1, 0.01))
plt.axvline(filtered_activations[filtered_activations>0].mean(-1), color='k')
plt.axvline(activation_thresh, color='r')

filtered_activations[filtered_activations<activation_thresh] = np.nan 
filtered_activations[-1, np.isnan(filtered_activations).all(0)] = 0

labels_predicted = np.nanargmax(filtered_activations, axis=0)

start_idx = 0
end_idx = start_idx+10

fig, axx = plt.subplots(3,1, sharex=True, figsize=(12,6))
plt.sca(axx[0])
plt.imshow(emg_test[:, start_idx*28:end_idx*28], aspect='auto')
plt.vlines(np.arange(end_idx-start_idx)*28, *plt.ylim(), color='r')

plt.sca(axx[1])
plt.imshow(filtered_activations[:, start_idx*28:end_idx*28], vmin=0, vmax=1, aspect='auto', interpolation='nearest')
plt.vlines(np.arange(end_idx-start_idx)*28, *plt.ylim(), color='r')
# plt.grid(True)
plt.gca().invert_yaxis()
plt.yticks(np.arange(11), list(range(10))+['NOGO'])

plt.sca(axx[2])
plt.plot(labels_predicted[start_idx*28:end_idx*28])
plt.vlines(np.arange(end_idx-start_idx)*28, *plt.ylim(), color='r')
plt.grid(True)
plt.yticks(np.arange(11), list(range(10))+['NOGO'])


# Numpy pseudocode for continuous inference

In [ ]:
model.layers

In [ ]:
model.summary(expand_nested=True)
plot_model(model)

In [ ]:
for l in model.layers:
    print('~'*50)
    print(l.name)
    for w in l.get_weights():
        print(w.shape)
    print('\n')

In [ ]:
rnn_in_w, rnn_h_w, rnn_b = model.layers[1].get_weights()
head_w, head_b = model.layers[-1].get_weights()

rnn_in_w.shape, rnn_h_w.shape, rnn_b.shape, head_w.shape, head_b.shape


## Feedforward NN mechanics

## Sliding inference

In [ ]:
activations = np.zeros((N_CLASSES, emg_test.shape[-1]))
print(emg_test.shape, activations.shape)

rnn_hidden = np.zeros(N_HIDDEN)

alpha=0.5

prev_activations = np.zeros(N_CLASSES);

for i, input_vector in tqdm(enumerate(emg_test.T), total=len(emg_test.T)):
    
    # first layer == Input layer
    output = input_vector
    
    # second layer == SimpleRNN
    rnn_hidden = keras.activations.relu(np.dot(output, rnn_in_w) + np.dot(rnn_hidden, rnn_h_w) + rnn_b)
    
    # third layer, Head == Dense
    output = np.dot(rnn_hidden, head_w) + head_b
    manual_activations = keras.activations.sigmoid(tf.convert_to_tensor([output])).numpy()[0]
    
    activations[:, i] = manual_activations.copy()
    
    activations[:, i] = manual_activations[:]*alpha + prev_activations*(1-alpha)
    prev_activations[:] = activations[:, i]    
    
#     if i>10000:
#         break


In [ ]:
filtered_activations = activations.copy()

activation_thresh = (filtered_activations[filtered_activations>0].mean(-1) + 2*filtered_activations[filtered_activations>0].std(-1)).mean()

plt.figure()
plt.hist(filtered_activations[filtered_activations>0], bins = np.arange(0, 1, 0.01))
plt.axvline(filtered_activations[filtered_activations>0].mean(-1), color='k')
plt.axvline(activation_thresh, color='r')

filtered_activations[filtered_activations<activation_thresh] = np.nan 
filtered_activations[-1, np.isnan(filtered_activations).all(0)] = 0

labels_predicted = np.nanargmax(filtered_activations, axis=0)

start_idx = 0
end_idx = start_idx+10

fig, axx = plt.subplots(3,1, sharex=True, figsize=(12,6))
plt.sca(axx[0])
plt.imshow(emg_test[:, start_idx*28:end_idx*28], aspect='auto')
plt.vlines(np.arange(end_idx-start_idx)*28, *plt.ylim(), color='r')

plt.sca(axx[1])
plt.imshow(filtered_activations[:, start_idx*28:end_idx*28], vmin=0, vmax=1, aspect='auto', interpolation='nearest')
plt.vlines(np.arange(end_idx-start_idx)*28, *plt.ylim(), color='r')
plt.grid(True)
plt.gca().invert_yaxis()
plt.yticks(np.arange(11), list(range(10))+['NOGO'])

plt.sca(axx[2])
plt.plot(labels_predicted[start_idx*28:end_idx*28])
plt.vlines(np.arange(end_idx-start_idx)*28, *plt.ylim(), color='r')
plt.grid(True)
plt.yticks(np.arange(11), list(range(10))+['NOGO'])


In [ ]:
fig, axx = plt.subplots(1,2, sharex=True, sharey=True, figsize=(12,6))
plt.sca(axx[0])
plt.imshow(activations_keras[:, :1000], 
           aspect='auto', 
           interpolation="nearest", vmin=0, vmax=1)
plt.title('Keras calculation')
plt.colorbar(orientation='horizontal')

plt.sca(axx[1])
plt.imshow(activations[:, :1000], 
           aspect='auto', 
           interpolation="nearest", vmin=0, vmax=1)
plt.colorbar(orientation='horizontal')

plt.title('Manual calculation')

### Check coincidence

In [ ]:
np.abs(activations - activations_keras).max()

## Numpy pseudocode

### DOT product (Скалярное умножение)

In [ ]:
x = emg_test[:,0].copy()
print(x.shape, rnn_in_w.shape)

np.dot(x, rnn_in_w)

In [ ]:
np.sum([x[i] * rnn_in_w[i, 0] for i in range(N_CHANNELS)])

In [ ]:
np.array([np.sum([x[i] * rnn_in_w[i, k] 
                  for i in range(N_CHANNELS)]) 
          for k in range(N_HIDDEN)])

In [ ]:
from numba import njit

@njit
def manual_relu(x):
    return max(x, 0)

@njit
def manual_sigmoid(x):
    return 1/(1+np.exp(-x))
        
@njit
def manual_dot(x, y, shape):
    s = 0.0
    for i in range(shape):
        s = s + x[i]*y[i]
    return s

rnn_hidden = np.zeros(N_HIDDEN)

def model_call(x, y):   
    """assume x is input data, y are activations for classes"""
    # first layer == Input layer
    x = x
    # y = np.zeros(N_CLASSES)
    
    # second layer == SimpleRNN
    rnn_hidden_prev = np.zeros(N_HIDDEN)
    for hi in range(N_HIDDEN):
        rnn_hidden_prev[hi] = manual_dot(x, rnn_in_w[:, hi], N_CHANNELS) + manual_dot(rnn_hidden, rnn_h_w[:, hi], N_HIDDEN) + rnn_b[hi]

    for hi in range(N_HIDDEN):
        rnn_hidden[hi] = manual_relu(rnn_hidden_prev[hi])
        
    for ci in range(N_CLASSES):
        y[ci] = manual_dot(rnn_hidden, head_w[:, ci], N_HIDDEN) + head_b[ci]

        
    for ci in range(N_CLASSES):
        y[ci] = manual_sigmoid(y[ci])
    
prev_x = np.zeros(N_CLASSES)

def postprocess(x, alpha):
    """assume x is an activation vec"""
    for ci in range(N_CLASSES):
        x[ci] = x[ci]*alpha + prev_x[ci]*(1-alpha) 
        prev_x[ci] = x[ci]    

In [ ]:
%%time

activations = np.zeros((N_CLASSES, emg_test.shape[-1]))
print(emg_test.shape, activations.shape)

act = np.zeros(N_CLASSES)

for i in tqdm(range(emg_test.shape[-1]), total = emg_test.shape[-1]):
    model_call(emg_test[:, i], act)   
    postprocess(act, 0.5)
    activations[:, i] = act[:]

    # break
    if (i>10000):
        break

In [ ]:
np.abs(activations[:, :10000] - activations_keras[:, :10000]).max()

In [ ]:
fig, axx = plt.subplots(1,2, sharex=True, sharey=True, figsize=(12,6))
plt.sca(axx[0])
plt.imshow(activations_keras[:, :10000], 
           aspect='auto', 
           interpolation="nearest", vmin=0, vmax=1)
plt.title('Keras calculation')
plt.colorbar(orientation='horizontal')

plt.sca(axx[1])
plt.imshow(activations[:, :10000], 
           aspect='auto', 
           interpolation="nearest", vmin=0, vmax=1)
plt.colorbar(orientation='horizontal')

plt.title('Manual calculation')

In [ ]:
filtered_activations = activations.copy()

activation_thresh = (filtered_activations[filtered_activations>0].mean(-1) + 3*filtered_activations[filtered_activations>0].std(-1)).mean()

plt.figure()
plt.hist(filtered_activations[filtered_activations>0], bins = np.arange(0, 1, 0.01))
plt.axvline(filtered_activations[filtered_activations>0].mean(-1), color='k')
plt.axvline(activation_thresh, color='r')

filtered_activations[filtered_activations<activation_thresh] = np.nan 
filtered_activations[-1, np.isnan(filtered_activations).all(0)] = 0

labels_predicted = np.nanargmax(filtered_activations, axis=0)

start_idx = 0
end_idx = start_idx+10

fig, axx = plt.subplots(3,1, sharex=True, figsize=(12,6))
plt.sca(axx[0])
plt.imshow(emg_test[:, start_idx*28:end_idx*28], aspect='auto')
plt.vlines(np.arange(end_idx-start_idx)*28, *plt.ylim(), color='r')

plt.sca(axx[1])
plt.imshow(filtered_activations[:, start_idx*28:end_idx*28], vmin=0, vmax=1, aspect='auto', interpolation='nearest')
plt.vlines(np.arange(end_idx-start_idx)*28, *plt.ylim(), color='r')
plt.grid(True)
plt.gca().invert_yaxis()
plt.yticks(np.arange(11), list(range(10))+['NOGO'])

plt.sca(axx[2])
plt.plot(labels_predicted[start_idx*28:end_idx*28])
plt.vlines(np.arange(end_idx-start_idx)*28, *plt.ylim(), color='r')
plt.grid(True)
plt.yticks(np.arange(11), list(range(10))+['NOGO'])


# Porting

Jinja - template rendering engine

https://jinja.palletsprojects.com/en/3.1.x/

`%conda install -c conda-forge jinja`

In [ ]:
# rnn_in_w, rnn_h_w, rnn_b = model.layers[1].get_weights()
# head_w, head_b = model.layers[-1].get_weights()

In [ ]:
rnn_in_w.shape, rnn_h_w.shape, rnn_b.shape, head_w.shape, head_b.shape

In [ ]:
import os
import jinja2

template_dir = './'
render_dir = './mnist_rnn_port/'

render_dict = {}
render_dict['N_CHANNELS'] = N_CHANNELS
render_dict['N_CLASSES'] = N_CLASSES
render_dict['N_HIDDEN'] = N_HIDDEN
render_dict['ACTIVATION_THRESH'] = activation_thresh
render_dict['ALPHA_ACTIVATIONS'] = alpha

render_dict['rnn_in_w'] = rnn_in_w
render_dict['rnn_h_w'] = rnn_h_w
render_dict['rnn_b'] = rnn_b
render_dict['head_w'] = head_w
render_dict['head_b'] = head_b

# print(render_dict)

render_filenames = ['w.cpp.jinja', 'w.h.jinja']
for filename in render_filenames:
    filename_out = os.path.join(render_dir, filename.replace('.jinja', ''))
    print(filename, '>', filename_out)
    with open(os.path.join(template_dir, filename), 'r') as f:
        render = jinja2.Template(f.read()).render(render_dict)

        print('~'*100)
        # print(render)
        
        with open(filename_out, 'w+') as f_out:
            f_out.write(render)

### Export toy data

In [ ]:
emg_test.shape

In [ ]:
DATA_LEN = 28*10

render_dict = {}
render_dict['DATA_LEN'] = DATA_LEN
render_dict['DATA'] = (emg_test[:, :DATA_LEN]*100).astype(int)

render_filenames = ['data.cpp.jinja', 'data.h.jinja']
for filename in render_filenames:
    filename_out = os.path.join(render_dir, filename.replace('.jinja', ''))
    print(filename, '>', filename_out)
    with open(os.path.join(template_dir, filename), 'r') as f:
        render = jinja2.Template(f.read()).render(render_dict)

        print('~'*100)
        print(render)

        with open(filename_out, 'w+') as f_out:
            f_out.write(render)

In [ ]:
render_dict['DATA'].max()

In [ ]:
plt.figure()
plt.imshow(render_dict['DATA'], aspect='auto')
plt.colorbar()

In [ ]:
emg_test[:, :DATA_LEN]

In [ ]:
emg_test[:, :DATA_LEN][0]

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA()
X = np.random.randn(100, 10)
pca.fit(X)

In [ ]:
%pip install micromlgen

In [ ]:
from micromlgen import port

In [ ]:
port(pca)

In [ ]:
print(port(pca))